In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import dash_table

import numpy as np
import pandas as pd
import math
from scipy.optimize import fsolve
from scipy.optimize import fmin
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
from os import listdir
%matplotlib inline

import base64
import datetime
import io

from textwrap import dedent as d
from scipy import stats
from decimal import Decimal

import plotly.graph_objs as go

In [2]:
#Constants
pi = math.pi
heV = 4.14e-15 #eV*s
c = 2.99792e8 #m/s
kbeV = 8.6173e-5 #eV/K
keV = 8.6173e-5 #eV/K
h = 6.626e-34
kb = 1.38065e-23
q = 1.60218e-19
k = 1.3806488e-23
T = 300

CellAreacm = 0.0453
CellArea = CellAreacm*10**-4 #m^2
Ps=100 #mW/cm^2

In [3]:
app=dash.Dash()

In [4]:
abs_layout = go.Layout(
    xaxis={'title': 'Vg (V)'},
    yaxis={'title': '|Id| (A)'},
    height=350,
    margin=go.Margin(
        l=35,
        r=5,
        b=40,
        t=5),
    dragmode='select',
    legend=dict(
        borderwidth=2,
        xanchor='right')
)

C:\Users\Joshua Wen-Kai Huang\Anaconda3\lib\site-packages\plotly\graph_objs\_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




In [5]:
app.layout = html.Div(children=[
    html.H1(children='OPV Analysis'),
    html.Div(children='''Blah'''),
    dcc.Upload(id = 'upload-data',children = html.Button('Upload File')),
    
    html.Div([
        html.Div([
            html.H3('Vg Selection'),    
            html.Div('Select the range of Vg, where the curve is linear, using the Box Select Tool below:'),
            dcc.Graph(
                id='transfer-curve',
                figure=go.Figure(),
                selectedData={'points': [], 'range': None}
            )
        ], style={'width': '50%', 'display': 'inline-block'}),
        
    ],
        style = {'width': '100%',
                'display': 'inline-block'}
    ),
])

In [6]:
@app.callback(Output('transfer-curve', 'figure'),
             [Input('upload-data', 'contents')])
def display_uploaded_data(contents):
    Ldata = pd.read_csv(contents, delimiter='\t', header=None)
    idx_end = Ldata[Ldata.iloc[:,0] == 'Jsc:'].index[0]
    Ldata = Ldata.iloc[:idx_end-1,:]
    Ldata.iloc[:,0] = pd.to_numeric(Ldata.iloc[:,0])
    Ldata.iloc[:,0]
    Ldata = np.array(Ldata)

    Ldata = np.insert(Ldata, 2, -Ldata[:,1], axis=1)

    JVinterp = interp1d(Ldata[:,0], Ldata[:,2], kind='cubic', bounds_error=False, fill_value='extrapolate')

    JscL = -JVinterp(0)
    VocL = fsolve(JVinterp,.95*max(Ldata[:,0]))
    PPV = fmin(lambda x: x*JVinterp(x),.8*VocL,disp=False)
    PCE = -PPV*JVinterp(PPV)
    FF = PCE/(JscL*VocL)*100

    #Sort data by ascending V, so less confusing
    Ldata = Ldata[Ldata[:,0].argsort()]

    jsc_idx1 = np.argmin(abs(Ldata[:,0]))
    jsc_idx2 = np.argmin(abs(Ldata[:,0]-0.05*VocL))
    voc_idx1 = np.argmin(abs(Ldata[:,0]-.95*VocL))
    voc_idx2 = np.argmin(abs(Ldata[:,0]-VocL))

    p1 = np.polyfit(Ldata[jsc_idx1:jsc_idx2,2],Ldata[jsc_idx1:jsc_idx2,1],1)
    RshL = p1[0]
    p2 = np.polyfit(Ldata[voc_idx1:voc_idx2,2],Ldata[voc_idx1:voc_idx2,1],1)
    RsL = p2[0]


    default_figsize = mpl.rcParamsDefault['figure.figsize']
    mpl.rcParams['figure.figsize'] = [1.5*val for val in default_figsize]
    font = {'family' : 'DejaVu Sans',
            'weight' : 'bold',
            'size'   : 22}

    mpl.rc('font', **font)
    mpl.rc('axes', linewidth=3)

    datas = [PCE, VocL, JscL, FF]
    n_rows = len(datas)
    rows = ['$PCE\ [\%]$', '$V_{OC}\ [V]$', '$J_{SC}\ [mA/cm^2]$', '$FF\ [\%]$']
    cell_text = []
    for row in range(n_rows):
        if row != 1:
            cell_text.append(['%1.1f' % datas[row]])
        else:
            cell_text.append(['%1.2f' % datas[row]])
 
    a_layout = abs_layout
    
    return go.Figure(
        data=[
            go.Scatter(
                x=Ldata[:,0],
                y=Ldata[:,2],
                mode='lines+markers'
            )
        ],
        layout=a_layout)


In [ ]:
if __name__ == '__main__':
    app.run_server()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Jul/2019 11:27:52] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2019 11:27:52] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.0.0&m=1563309670 HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2019 11:27:52] "GET /_dash-component-suites/dash_core_components/highlight.pack.js?v=1.0.0&m=1563309546 HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2019 11:27:52] "GET /_dash-component-suites/dash_renderer/dash_renderer.min.js?v=1.0.0&m=1563309670 HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2019 11:27:52] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.0.0&m=1563309670 HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2019 11:27:53] "GET /_dash-component-suites/dash_table/bundle.js?v=4.0.2&m=1563309547 HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2019 11:27:53] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.0.0&m=1563309670 HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2019 11:27:53] "GET /_dash-compone

127.0.0.1 - - [29/Jul/2019 11:28:17] "POST /_dash-update-component HTTP/1.1" 500 -


In [ ]:
#Upload data --> Process data using OPV tool --> Plot processed data